# Optical Flow

## Sparse (Lucas Kanade) 
https://pysource.com/2018/05/14/optical-flow-with-lucas-kanade-method-opencv-3-4-with-python-3-tutorial-31/

In [1]:
import cv2
import numpy as np
import utils_flow as ut_fl
from pathlib import Path

In [2]:
camera = cv2.VideoCapture(0)

_, frame = camera.read()
old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Lucas kanade params
lk_params = dict(
    winSize=(10, 10),
    maxLevel=10,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
)


def select_point(event, x, y, flags, params):
    global point, point_selected, old_points

    if (event == cv2.EVENT_LBUTTONDOWN):
        point = (x, y)
        point_selected = True
        old_points = np.array([[x, y]], dtype=np.float32)

cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", select_point)

point_selected = False
point = ()
old_points = np.array([[]], dtype=np.float32)
old_points.astype(np.float32)



while True:
    _, frame = camera.read()

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if point_selected is True:
        cv2.circle(frame, point, 5, (0, 0, 255), 2)
        new_points, status, error = cv2.calcOpticalFlowPyrLK(
            old_gray, gray_frame, old_points, None, **lk_params
        )

        old_gray = gray_frame.copy()
        old_points = new_points

        x, y = new_points.ravel().astype(np.int32)

        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == 27:
        break

camera.release()
cv2.destroyAllWindows()

## Dense - Flow vector drawing for a entire video or camera

Parameters for optical flow

https://amroamroamro.github.io/mexopencv/matlab/cv.calcOpticalFlowFarneback.html


https://www.youtube.com/watch?v=WrlH5hHv0gE

https://github.com/niconielsen32/ComputerVision/blob/master/opticalFlow/denseOpticalFlow.py

In [3]:
# Camera test, set True to use camera as source
camera_test = False
test_path = str(Path('OTHER/video_test.mp4'))

In [6]:
optical_folder = 'OPTICAL'

if camera_test:
    video = 0
else:
    video = test_path

vidcap = cv2.VideoCapture(video)

ret, frame1 = vidcap.read()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

f_number = 1
while (True):
    ret, frame2 = vidcap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(
            prev=prvs,
            next=next,
            flow=None,
            pyr_scale=0.4,
            levels=1,
            winsize=15,
            iterations=1,
            poly_n=8,
            poly_sigma=1.1,
            flags=0
        )

    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    (h, s, v) = cv2.split(hsv)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    # test converting BGR to GRAYSCALE
    gr1 = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    # test using value from hsv
    gr2 = v

    cv2.imshow('Vector Flow', ut_fl.draw_flow(next, flow))

    # cv2.imshow('BGR Flow', bgr)
    # cv2.imshow('GR1 BGRTOGRAY', gr1)
    # cv2.imshow('GR2 SPLIT', gr2)

    # cv2.imwrite('{}/test_{}.jpg'.format(optical_folder,f_number),bgr)
    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break

    f_number = f_number + 1
    prvs = next

vidcap.release()
cv2.destroyAllWindows()